# Choral Explanations on Alkemio

At the [TIP knowledge base](https://alkem.io/tip/knowledge-base) on [Alkemio](https://welcome.alkem.io/) we apply [Choral Explanations](https://hapgood.us/2016/05/13/choral-explanations/) to develop knowledge commons in a Q&A format. In this project we explore new possibilities for navigation and visualisation.

## Copyright and license

### Code

Copyright: sander.dijkhuis@cleverbase.com 2024; Licensed under the EUPL.

### Example content

See content source.

In [1]:
import json
import os
import re
import shutil
import urllib
from datetime import datetime
from urllib.request import Request
from IPython.display import display, Markdown

os.environ["no_proxy"] = "*"

Use the [GraphQL Playground](https://alkem.io/graphql) to develop queries.

In [2]:
endpoint = "https://alkem.io/api/private/graphql"
reader = open("queries.graphql", "r")
queries = reader.read()
reader.close()

In [3]:
def query(operation, variables):
    payload = {
        "query": queries,
        "operationName": operation,
        "variables": variables
    }
    headers = {
        "Content-Type": "application/json"
    }
    body = json.dumps(payload).encode("utf-8")
    response = urllib.request.urlopen(Request(endpoint, body, headers))
    return json.loads(response.read().decode("utf-8"))

In [4]:
result = query("questions", { "spaceNameId": "tip" })

In [108]:
questions = []

def strip_license(markdown):
    return re.sub(r".*Bijdragen zijn gelicenseerd onder.*\[.*CC BY 4\.0.*\]\(https:\/\/creativecommons\.org\/licenses\/by\/4\.0\/deed\.nl\).*\..*", "", markdown.strip()).strip()

for question_data in result["data"]["space"]["collaboration"]["callouts"]:
    question = {
        "id": question_data["nameID"],
        "title": question_data["framing"]["profile"]["displayName"].strip(),
        "url": question_data["framing"]["profile"]["url"],
        "description_md": strip_license(question_data["framing"]["profile"]["description"]),
        "answers": []
    }
    by = question_data["createdBy"]
    if by != None:
        question["author_name"] = by["profile"]["displayName"]
        question["author_url"] = by["profile"]["url"]
    questions.append(question)
    for answer_data in question_data["contributions"]:
        answer = {
            "id": answer_data["post"]["nameID"],
            "title": answer_data["post"]["profile"]["displayName"].strip(),
            "url": answer_data["post"]["profile"]["url"],
            "author_name": answer_data["post"]["createdBy"]["profile"]["displayName"],
            "author_url": answer_data["post"]["profile"]["url"],
            "description_md": answer_data["post"]["profile"]["description"],
            "comments": []
        }
        question["answers"].append(answer)
        for comment_data in answer_data["post"]["comments"]["messages"]:
            comment = {
                "author_name": comment_data["sender"]["profile"]["displayName"],
                "author_url": comment_data["sender"]["profile"]["url"],
                "date": datetime.fromtimestamp(comment_data["timestamp"] / 1000),
                "content_md": comment_data["message"]
            }
            answer["comments"].append(comment)

In [133]:
def link(label, url):
    return f"[🏔️ {label}]({url})"

def linked_title(contribution):
    return link(contribution["title"], contribution["url"])

def linked_author(contribution):
    return link(contribution["author_name"], contribution["author_url"])

def quote(markdown):
    return "".join([f">{line}\n" for line in markdown.splitlines()])

def readable_date(contribution):
    return contribution["date"].strftime("%Y-%m-%d %H:%M UTC")

def indent(markdown):
    return "".join([f"  {line}\n" for line in markdown.splitlines()])

In [232]:
def replace_links(markdown, index):
    def replacement(match):
        label, url = match.groups()
        match = CALLOUT_URL.match(url)
        if match:
            dict = match.groupdict()
            question = dict['q1'] or dict['q2']
            question_title = index[(question,)]['question']
            answer = dict['a1']
            if answer:
                answer_title = index[(question, answer)]['answer']
                return f"[📌 {answer_title}]({question}.md#{answer})"
            else:
                return f"[📄 {question_title}]({question}.md)"
        else:
            return f"[🌐 {label}]({url})"
    return INLINE_LINK.sub(replacement, markdown)
display(replace_links("test [a](https://alkem.io/tip/collaboration/watisdetoegevoegd-5977) test2 [b](https://alkem.io/tip/collaboration/watisdekennisagen-9941/posts/kennisagenda-5711)", index))

'test [📄 Wat is de toegevoegde waarde van vertrouwensdiensten voor de digitale overheid?](watisdetoegevoegd-5977.md) test2 [📌 Kennisagenda](watisdekennisagen-9941.md#kennisagenda-5711)'

In [252]:
dir_path = "example/tip"
shutil.rmtree(dir_path)
os.makedirs(dir_path, exist_ok=True)

for question in questions:
    file_path = f"{dir_path}/{question['id']}.md"
    with open(file_path, "w") as file:
        file.write("[🏔️ Alkemio](https://welcome.alkem.io/) › [🏔️ TIP](https://alkem.io/tip/dashboard) › Kennisbank\n")
        file.write(f"# 📄 {question['title']}\n")
        file.write(replace_links(question["description_md"], index))
        file.write("\n")
        if "author_name" in question:
            file.write(f"> Oorspronkelijk gevraagd door {linked_author(question)}.")
        file.write(f" [`🏔️ Origineel`]({question['url']})\n\n")
        if len(question["answers"]) > 0:
            file.write("## Antwoorden\n")
        for answer in question["answers"]:
            file.write(f"- ### <a id=\"{answer['id']}\"></a> 📌 {answer['title']}\n")
            file.write(indent(replace_links(answer["description_md"], index)))
            file.write(f"\n  \n  > Oorspronkelijk geantwoord door {linked_author(answer)}.")
            file.write(f" [`🏔️ Origineel`]({answer['url']})\n\n")
            if len(answer["comments"]) > 0:
                file.write(indent("#### Reacties\n"))
            for comment in answer["comments"]:
                file.write(indent(indent(f"- ##### {linked_author(comment)} {readable_date(comment)}\n      ")))
                file.write(indent(indent(indent(replace_links(comment["content_md"], index)))))
        incoming = {}
        for src, props in index.items():
            if (question['id'],) in props['outgoing']:
                incoming[src] = props
        if len(incoming) > 0:
            file.write("## Verwijzingen naar deze vraag\n")
            for path, props in incoming.items():
                url = f"{path[0]}.md"
                if len(path) == 2:
                    url += f"#{path[1]}"
                label = props['question']
                if 'answer' in props:
                    label = f"📌 {label} {props['answer']}"
                else:
                    label = f"📄 {label}"
                file.write(f"- [{label}]({url})\n")
        file.write("* * *\n<small>Bijdragen zijn gelicenseerd onder [🌐 CC BY 4.0](https://creativecommons.org/licenses/by/4.0/deed.nl).</small>")
        file.write("\n")

In [101]:
INLINE_LINK = re.compile(r"\[([^\]]+)\]\(([^)]+)\)")
CALLOUT_URL = re.compile(r"""^https:\/\/alkem\.io\/tip\/co(?:(?:llaboration\/(?P<q1>[^/]+)(?:\/posts\/(?P<a1>[^/]+))?)|(?:ntribute\/callouts\/(?P<q2>[^/]+)))$""")
markdown = "  [IANAL](https://en.wikipedia.org/wiki/IANAL), maar dit is mijn huidige inzicht [🏔️ Wat zijn de juridische grenzen tussen Corporate Seal en Personal Qualified signature?](https://alkem.io/tip/collaboration/juridischegrenzent-2374). Om te beginnen, zie wat in eIDAS ([Wat is eIDAS?](https://alkem.io/tip/contribute/callouts/watiseidas-4062)) over rechtsgevolg staat [🏔️ Rechtsgevolg volgens eIDAS](https://alkem.io/tip/collaboration/juridischegrenzent-2374/posts/rechtsgevolgvolgens-1804) beschreven:"
def links(markdown):
    '''returns a dict of connected nodes (q,a) to edge labels'''
    links = {}
    for label, url in INLINE_LINK.findall(markdown):
        match = CALLOUT_URL.match(url)
        if match:
            dict = match.groupdict()
            question = dict['q1'] or dict['q2']
            answer = dict['a1']
            if answer:
                link = (question, answer)
            else:
                link = (question,)
            if link in links:
                links[link].append(label)
            else:
                links[link] = {label}
    return links
display(links(markdown))


{('juridischegrenzent-2374',): {'🏔️ Wat zijn de juridische grenzen tussen Corporate Seal en Personal Qualified signature?'},
 ('watiseidas-4062',): {'Wat is eIDAS?'},
 ('juridischegrenzent-2374',
  'rechtsgevolgvolgens-1804'): {'🏔️ Rechtsgevolg volgens eIDAS'}}

In [162]:
def make_index(questions):
    index = {}
    for question in questions:
        index[(question['id'],)] = {
            'question': question['title'],
            'outgoing': links(question['description_md'])
        }
        for answer in question['answers']:
            index[(question['id'], answer['id'])] = {
                'question': question['title'],
                'answer': answer['title'],
                'outgoing': links(answer['description_md'])
            }
    return index
index = make_index(questions)
display(list(index.items())[-20:-15])

[(('tiptoetsingskader-3432', 'categorieenintoets-6290'),
  {'question': 'Hoe toets ik een vertrouwensraamwerk?',
   'answer': 'categorieën in toetsing',
   'outgoing': {}}),
 (('tiptoetsingskader-3432', 'voorbeeldenuiteu-i-7828'),
  {'question': 'Hoe toets ik een vertrouwensraamwerk?',
   'answer': 'Voorbeelden uit EU-initiatieven op interoperabiliteit',
   'outgoing': {('hoetoetsikeenhan-831',): {'Hoe toets ik een handelingsomgeving tegen de afspraken van TIP?'}}}),
 (('hoetoetsikeenhan-831',),
  {'question': 'Hoe toets ik een handelingsomgeving tegen de afspraken van TIP?',
   'outgoing': {}}),
 (('hoetoetsikeenhan-831', 'beginmeteenprogra-2384'),
  {'question': 'Hoe toets ik een handelingsomgeving tegen de afspraken van TIP?',
   'answer': 'Begin met een Programma van Eisen',
   'outgoing': {('hoekunnenweeffect-1138',): {'Hoe kunnen we effectief publiek-privaat samenwerken aan een afsprakenstelsel?'},
    ('tiptoetsingskader-3432',): {'Hoe toets ik een vertrouwensraamwerk?'}}}),
 ((